In [1]:
import h5py
import scanpy as sc
import scipy
import mira
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

/media/RAIDArray/Nick/miniconda3/envs/mira-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

# SVM Human Tonsil Atlas classification

In [3]:
adata_donor = sc.read_h5ad('/media/RAIDArray/JingyuFan/projects/human_B_cell/scRNA_220830/human_B_cell_scRNA_seq_220831/outs/human_B_cell_scRNA_seq_230327.umap.leiden_clusters.h5ad')

# Check aggregated file
adata_donor

AnnData object with n_obs × n_vars = 25237 × 20921
    obs: 'Classification', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic_13', 'topic_14', 'topic_15', 'topic_16', 'topic_17', 'topic_18', 'topic_19', 'leiden'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'exog', 'endog'
    uns: 'Classification_colors', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'topic_dendogram', 'umap'
    obsm: 'X_topic_compositions', 'X_umap', 'X_umap_features'
    varm: 'topic_feature_activations', 'topic_feature_compositions'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [4]:
adata_tonsil = sc.read_h5ad('/ix/cigcore/sbg57/multiome/tonsil_atlas/seurat_object.h5ad') # we convert 'annotation_figure_1' to 'Classifcation' in steps below

# Check aggregated file
adata_tonsil

/ix/djishnu/peasena/miniconda3/envs/mira-env/lib/python3.9/site-packages/anndata/compat/__init__.py:329: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(


AnnData object with n_obs × n_vars = 68749 × 2000
    obs: 'barcode', 'donor_id', 'gem_id', 'library_name', 'assay', 'sex', 'age', 'age_group', 'hospital', 'cohort_type', 'cause_for_tonsillectomy', 'is_hashed', 'preservation', 'nCount_RNA', 'nFeature_RNA', 'pct_mt', 'pct_ribosomal', 'pDNN_hashing', 'pDNN_scrublet', 'pDNN_union', 'scrublet_doublet_scores', 'S.Score', 'G2M.Score', 'Phase', 'scrublet_predicted_doublet', 'doublet_score_scDblFinder', 'annotation_level_1', 'annotation_level_1_probability', 'annotation_figure_1', 'annotation_20220215', 'annotation_20220619', 'annotation_20230508', 'annotation_20230508_probability', 'UMAP_1_level_1', 'UMAP_2_level_1', 'UMAP_1_20220215', 'UMAP_2_20220215', 'UMAP_1_20230508', 'UMAP_2_20230508', 'type', 'nCount_ATAC', 'nFeature_ATAC', 'RNA.weight', 'ATAC.weight', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'

In [8]:
adata_tonsil.obs['type'].unique()

array(['discovery_multiome', 'validation_multiome'], dtype=object)

In [31]:
# processed object already has mapping tags to transfer Classification
adata_tonsil_1 = sc.read_h5ad('/ix/cigcore/sbg57/multiome/tonsil_atlas/bc_tonsil_processed.h5ad')

# Check aggregated file
adata_tonsil_1.obs.columns

Index(['batch', 'n_genes', 'n_genes_by_counts', 'total_counts',
       'total_counts_mt', 'pct_counts_mt', 'doublet_score',
       'predicted_doublet', 'leiden', 'Tag', 'Classification',
       'leiden_res_0.10', 'leiden_res_0.25', 'leiden_res_0.50', 'modality'],
      dtype='object')

In [32]:
# annotations from 'annotation_figure_1' - coarse annotations
adata_tonsil_1.obs['Classification'].unique()

['NBC', 'GCBC', 'MBC', 'PC', 'Activated NBC']
Categories (5, object): ['Activated NBC', 'GCBC', 'MBC', 'NBC', 'PC']

In [33]:
adata_tonsil_1

AnnData object with n_obs × n_vars = 21699 × 25600
    obs: 'batch', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'doublet_score', 'predicted_doublet', 'leiden', 'Tag', 'Classification', 'leiden_res_0.10', 'leiden_res_0.25', 'leiden_res_0.50', 'modality'
    var: 'gene_ids', 'feature_types', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'batch_colors', 'dendrogram_leiden_res_0.10', 'hvg', 'leiden', 'leiden_colors', 'leiden_res_0.10', 'leiden_res_0.10_colors', 'leiden_res_0.25', 'leiden_res_0.25_colors', 'leiden_res_0.50', 'leiden_res_0.50_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [34]:
# Add a new column "Classification" and extract everything after the last '_'
adata_tonsil_1.obs['Tag'] = adata_tonsil_1.obs.index.str.split('_').str[-1]

# Confirm the addition
print(adata_tonsil_1.obs.head())

                                                       batch  n_genes  \
in_vivo_male_child_AAACAGCCAGTAGGTG-1  in_vivo_male_child_1a     1683   
in_vivo_male_child_AAACAGCCATTATGCG-1  in_vivo_male_child_1a     1599   
in_vivo_male_child_AAACAGCCATTTAAGC-1  in_vivo_male_child_1a     3129   
in_vivo_male_child_AAACATGCAAATTCGT-1  in_vivo_male_child_1a     2081   
in_vivo_male_child_AAACATGCAACGTGCT-1  in_vivo_male_child_1a     1569   

                                       n_genes_by_counts  total_counts  \
in_vivo_male_child_AAACAGCCAGTAGGTG-1               1682        3270.0   
in_vivo_male_child_AAACAGCCATTATGCG-1               1599        2955.0   
in_vivo_male_child_AAACAGCCATTTAAGC-1               3129        8203.0   
in_vivo_male_child_AAACATGCAAATTCGT-1               2080        4358.0   
in_vivo_male_child_AAACATGCAACGTGCT-1               1568        3095.0   

                                       total_counts_mt  pct_counts_mt  \
in_vivo_male_child_AAACAGCCAGTAGGTG-1      

In [35]:
# this step uses cell index tags/batch to transfer the Classfication column from seurat object
# Ensure required columns exist in both datasets
if all(col in adata_tonsil_1.obs.columns for col in ['Tag', 'batch']) and \
   all(col in adata_tonsil.obs.columns for col in ['Tag', 'donor_id', 'annotation_figure_1']):
    
    # Create a mapping for rows in adata_invivo that match the donor_id condition
    annotation_map = adata_tonsil_1.obs.loc[
        adata_tonsil_1.obs['donor_id'] == 'BCLL-8-T', ['Tag', 'annotation_figure_1']
    ].set_index('Tag')['annotation_figure_1'].to_dict()

    # Apply mapping to the Classification column in adata
    adata_tonsil.obs['Classification'] = adata_tonsil.obs.apply(
        lambda row: annotation_map[row['Tag']] 
        if row['batch'] == 'in_vivo_male_child_1a' and row['Tag'] in annotation_map 
        else row.get('Classification', None), 
        axis=1
    )

    # Display rows updated for validation
    print(adata_tonsil_1.obs.loc[adata_tonsil_1.obs['Classification'].notna()])
else:
    print("Required columns are missing in one of the objects.")


Required columns are missing in one of the objects.


In [36]:
adata_tonsil_1.obs.columns

Index(['batch', 'n_genes', 'n_genes_by_counts', 'total_counts',
       'total_counts_mt', 'pct_counts_mt', 'doublet_score',
       'predicted_doublet', 'leiden', 'Tag', 'Classification',
       'leiden_res_0.10', 'leiden_res_0.25', 'leiden_res_0.50', 'modality'],
      dtype='object')

In [37]:
# Assuming adata_invitro and adata_invivo are your two datasets
sc.pp.highly_variable_genes(adata_donor)
sc.pp.highly_variable_genes(adata_tonsil_1)

# Use only highly variable genes in both datasets
adata_donor = adata_donor[:, adata_donor.var.highly_variable]
adata_tonsil_1 = adata_tonsil_1[:, adata_tonsil_1.var.highly_variable]

AttributeError: 'DataFrame' object has no attribute 'highly_variable'

In [38]:
# Get the common genes between the two datasets
common_genes = adata_donor.var_names.intersection(adata_tonsil_1.var_names)

# Subset both datasets to have the same features
adata_donor = adata_donor[:, common_genes]
adata_tonsil_1 = adata_tonsil_1[:, common_genes]

In [39]:
adata_tonsil_1

View of AnnData object with n_obs × n_vars = 21699 × 22482
    obs: 'batch', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'doublet_score', 'predicted_doublet', 'leiden', 'Tag', 'Classification', 'leiden_res_0.10', 'leiden_res_0.25', 'leiden_res_0.50', 'modality'
    var: 'gene_ids', 'feature_types', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'batch_colors', 'dendrogram_leiden_res_0.10', 'hvg', 'leiden', 'leiden_colors', 'leiden_res_0.10', 'leiden_res_0.10_colors', 'leiden_res_0.25', 'leiden_res_0.25_colors', 'leiden_res_0.50', 'leiden_res_0.50_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [40]:
adata_tonsil = adata_tonsil_1

In [14]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Extract data and labels from AnnData objects
X_train = adata_tonsil.X.toarray() if not isinstance(adata_tonsil.X, np.ndarray) else adata_tonsil.X
y_train = adata_tonsil.obs["Classification"]  # Replace with the correct label column
X_test = adata_donor.X.toarray() if not isinstance(adata_donor.X, np.ndarray) else adata_donor.X
#y_test = adata_donor.obs["leiden"]  # don't need this

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train SVM with probability=True
# linear works (change to rbf if need be)
svm = SVC(kernel="linear", probability=True, random_state=42)
svm.fit(X_train_scaled, y_train)

# Predict probabilities on the test set
probabilities = svm.predict_proba(X_test_scaled)

# Set a threshold
threshold = 0.01  # Confidence level
predicted_labels = svm.predict(X_test_scaled)
max_probabilities = probabilities.max(axis=1)

# Replace low-confidence predictions with "NA"
final_predictions = [
    label if max_prob >= threshold else "NA"
    for label, max_prob in zip(predicted_labels, max_probabilities)
]

# Convert probabilities to a pandas DataFrame
probabilities_df = pd.DataFrame(
    probabilities,
    columns=[f"prob_{cls}" for cls in svm.classes_],  # Column names based on classes
    index=adata_donor.obs.index  # Ensure alignment with AnnData index
)

# Add the final predictions to the adata_donor.obs
adata_donor.obs["predicted_labels"] = final_predictions
adata_donor.obs = pd.concat([adata_donor.obs, probabilities_df], axis=1)

# Visualize or analyze the predictions
print(adata_donor.obs[["leiden", "predicted_labels"]].head())

/scratch/slurm-1081423/ipykernel_440/3392752344.py:43: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_donor.obs["predicted_labels"] = final_predictions


                   leiden predicted_labels
AAACCTGAGAATAGGG-1      1    Activated NBC
AAACCTGAGACAGACC-1      6              MBC
AAACCTGAGACCTAGG-1      1    Activated NBC
AAACCTGAGACGCAAC-1      1             GCBC
AAACCTGAGAGCTATA-1      0               PC


In [ ]:
adata_donor.write("/ix/cigcore/sbg57/multiome/donor2_stanford/241017_donor2_stanford_agg_nosecondary/outs/scRNA_male_scimilarity_corrected.h5ad")